### Pre-requisite - Download Datasets
Run all the cells below after downloading the required kaggle.json file (follow [these steps](https://www.kaggle.com/discussions/general/156610))

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!ls -lha kaggle.json

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!pwd

In [ ]:
!kaggle datasets download -d anupriyakkumari/instagram-5-classes-dataset-1

In [ ]:
!kaggle datasets download -d anupriyakkumari/instagram-5-classes-dataset-2

In [ ]:
!unzip instagram-5-classes-dataset-1.zip

In [ ]:
!unzip instagram-5-classes-dataset-2.zip

* Note - rename the folders for consistency.
We renamed them to - Instagram_Dataset_1 and Instagram_Dataset_2

#1. Transfer Learning Model - Xception

### 1.1 Train model

The following approach uses transfer learning on base model Xception. We need to run each cell one by one. Importing these libraries is first step.
Then we need our dataset divided into train, validation and test folders (each with subfolders of 5 classes) and uploaded on colab (either by mounting drive or downloading directly from kaggle in the colab notebook). The paths of each dataset directory can be changed as needed.



In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

Here, we are uploading our files from each of the folders and subfolders, setting the parameters as required for the model and printing the class names, creating a dictionary to access class names, and printing number of batches in each folder - train, validation and test

In [ ]:
dir_train = "/content/Instagram_Dataset_1/classes"
dir_new="/content/Instagram_Dataset_2/classes"
#using smaller dataset for now
train_ds=tf.keras.utils.image_dataset_from_directory(
    dir_train,
    color_mode="rgb",
    image_size=(150, 150),
    validation_split=0.1,
    subset="training",
    shuffle=True,
    seed=2,
    batch_size=64
    )

validation_ds=tf.keras.utils.image_dataset_from_directory(
    dir_train,
    color_mode="rgb",
    image_size=(150, 150),
    validation_split=0.1,
    subset="validation",
    shuffle=True,
    seed=2,
    batch_size=64
    )
directory_test1="/content/Instagram_Dataset_1/test"
directory_test2="/content/Instagram_Dataset_2/test"
test_ds=tf.keras.utils.image_dataset_from_directory(
    directory_test1,
    color_mode="rgb",
    image_size=(150, 150),
    shuffle=True,
    seed=2,
    batch_size=64
    )
class_names = train_ds.class_names
print(class_names)
#class_names_dic1={0:'animals',1:'beauty',2:'food',3:'memes',4:'travel'}
class_names_dic2={0:'beauty',1:'food',2:'memes',3:'pets',4:'travel'}

print("Number of training samples: %d" % tf.data.experimental.cardinality(train_ds))
print(
    "Number of validation samples: %d" % tf.data.experimental.cardinality(validation_ds)
)
print("Number of test samples: %d" % tf.data.experimental.cardinality(test_ds))

Now, by running this cell, we get the shape of the image and label as it gets uploaded using tf.keras.utils.image_dataset_from_directory first as batches and in those batches are images (float32 tensor) and labels (int32 tensor). We created batches of 10.

In [ ]:
for image, label in train_ds.take(1):
  #print element in the tuple of train_ds which has 2 tuples inside indicating (batchsize,height,width,channels) and (batchsize,)
    print(image.shape, label.shape)
    for image, label in zip(image, label):
      #each batch further has 64 images each in it stored in zip(image,label)
        print(image.shape, label)


By running the cell below, we plot the 10 images from one batch randomly in a 2x5 plot.

In [ ]:
import matplotlib.pyplot as plt
class_names = list(train_ds.class_names)
# Create a figure with 2 rows and 5 columns of subplots
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(10,5))

# Flatten the axes array to simplify indexing
axes = axes.flatten()
for images, labels in train_ds:
  # Plot each image on a subplot
  for i in range(len(images)):
    axes[i].imshow(images[i].numpy().astype('uint8'))
    axes[i].set_title(class_names_dic2[(labels[i].numpy())])
    axes[i].axis('off')
# Show the plot
plt.show()

Here we are creating a  data augmentation layer

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1), layers.RandomContrast(factor=0.2),
]
)

Here we are visualizing the first augmented image from the first batch and showing the variation produced on a 9x9 plot

In [ ]:
import numpy as np
for images, labels in train_ds.take(1):
    plt.figure(figsize=(10, 10))
    first_image = images[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(
            tf.expand_dims(first_image, 0), training=True
        )
        plt.imshow(augmented_image[0].numpy().astype("int32"))
        plt.title(int(labels[0]))
        plt.axis("off")


Now we get a base model, Xception (trained on Imagenet) and proceed to freeze it.

In [ ]:
base_model = keras.applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,)
  # Do not include the ImageNet classifier at the top as we don't need those classes

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(150, 150, 3))
x = data_augmentation(inputs)  # Apply random data augmentation from the layer we created above

# Pre-trained Xception weights requires that input be scaled
# from (0, 255) to a range of (-1., +1.), the rescaling layer
# outputs: `(inputs * scale) + offset`
# we create a rescaling layer and apply it to "x" which already has data augmentation applied to it
scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(x)

# The base model contains batchnorm layers. We want to keep them in inference mode (moving forward and not changing any weights/biases yet, only giving result)
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here, hence training is false
x = base_model(x, training=False)
# using pooling layer
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

epochs =3
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

In [ ]:
# Unfreeze the base_model. Note that it keeps running in inference mode
# since we passed `training=False` when calling it. This means that
# the batchnorm layers will not update their batch statistics.
# This prevents the batchnorm layers from undoing all the training
# we've done so far.
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-4),  # Low learning rate
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

epochs = 5
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

### 1.2 Save model

In [ ]:
model.save("/content/gdrive/MyDrive/Image_Classification/model_xception.h5")

### 1.3 Test model

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print('Test accuracy: ',test_acc)
print('Test loss: ',test_loss)

In [ ]:
# using the xception model to display images on unseen data with predicted labels - very inaccuarate, as expected.
from keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.utils import load_img
from tensorflow.keras.preprocessing import image
from keras.models import load_model
import numpy as np
import os
import matplotlib.pyplot as plt

model = load_model('/content/gdrive/MyDrive/Image_Classification/model_xception.h5')

# Set the path to the folder containing the images
folder_path = '/content/Instagram_Dataset_1/unseen'

# Loop through the images in the folder
for filename in os.listdir(folder_path):

    # Load the image
    img = image.load_img(os.path.join(folder_path, filename), target_size=(180, 180))

    # Convert the image to a numpy array
    img_array = image.img_to_array(img)

    # Reshape the array to match the input shape of the VGG16 model
    img_array = np.expand_dims(img_array, axis=0)

    # Preprocess the input image (normalize pixel values to be between -1 and 1)
    img_array = preprocess_input(img_array)

    # Make a prediction on the image
    preds = model.predict(img_array)
    print(preds)

    # Decode the prediction into a human-readable label
    label = int(preds.argmax(axis=-1))
    label_name={0:'beauty',1:'food',2:'memes',3:'pets',4:'travel'}


    # Display the image with predicted label
    plt.imshow(img)
    plt.title(label_name[label])
    plt.show()



#2. Transfer Learning Model - MobileNetV2


### 2.1 Train model

In this approach, we again use a base model, MobileNetV2 and proceed as in the first approach. Data loading and preprocessing is similar to the second approach with some small changes.

In [ ]:
#loading essential libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
#setting important parameters and loading the three required datasets
batch_size = 64
img_height = 160
img_width = 160
data_dir= "/content/Instagram_Dataset_1/classes"
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.1,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
class_names = train_ds.class_names
print(class_names)


In [ ]:
#creating test_ds and loading images
batch_size = 32
img_height = 160
img_width = 160
test_dir= "/content/Instagram_Dataset_1/test"
test_ds = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  image_size=(img_height, img_width),
  batch_size=batch_size)


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomContrast(factor=0.3),

])

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SIZE = (160, 160)
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
image_batch, label_batch = next(iter(train_ds))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [ ]:
base_model.trainable = False

In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
len(model.trainable_variables)

In [ ]:
initial_epochs = 5

loss0, accuracy0 = model.evaluate(val_ds)

In [ ]:
history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=val_ds)

In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
              metrics=['accuracy'])

In [ ]:
fine_tune_epochs = 5
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_ds,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=val_ds)

### 2.2 Save model

In [ ]:
model.save("/content/gdrive/MyDrive/Image_Classification/model_mobilenetv2.h5")

### 2.3 Test model

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print('Test accuracy :', accuracy)
print ('Test loss: ', loss)

#3. Transfer Learning Model - VGG16


### 3.1 Train model

In [ ]:
import os
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np

In [ ]:
BATCH_SIZE = 64

train_generator = ImageDataGenerator(rotation_range=90,
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5,
                                     height_shift_range=0.5,
                                     horizontal_flip=True,
                                     vertical_flip=True,
                                     validation_split=0.15,
                                     preprocessing_function=preprocess_input) # VGG16 preprocessing

test_generator = ImageDataGenerator(preprocessing_function=preprocess_input) # VGG16 preprocessing

In [ ]:
download_dir = Path('/content/Instagram_Dataset_1')


In [ ]:
train_data_dir = download_dir/'classes'
test_data_dir = download_dir/'test'

traingen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               subset='training',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

validgen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

testgen = test_generator.flow_from_directory(test_data_dir,
                                             target_size=(224, 224),
                                             class_mode=None,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)

In [ ]:
def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):
    """
    Compiles a model integrated with VGG16 pretrained layers

    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """

    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = VGG16(include_top=False,
                     weights='imagenet',
                     input_shape=input_shape)

    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(1072, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)

    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
input_shape = (224, 224, 3)
optim_1 = Adam(learning_rate=0.001)
n_classes=5

n_steps = traingen.samples // BATCH_SIZE
n_val_steps = validgen.samples // BATCH_SIZE
n_epochs = 6

# First we'll train the model without Fine-tuning
vgg_model = create_model(input_shape, n_classes, optim_1, fine_tune=0)

In [ ]:
!pip install livelossplot

In [ ]:
from livelossplot.inputs.keras import PlotLossesCallback

plot_loss_1 = PlotLossesCallback()

# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='tl_model_v1.weights.best.hdf5',
                                  save_best_only=True,
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',
                           patience=2,
                           restore_best_weights=True,
                           mode='min')

In [ ]:
%%time

vgg_history = vgg_model.fit(traingen,
                            batch_size=BATCH_SIZE,
                            epochs=n_epochs,
                            validation_data=validgen,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps,
                            callbacks=[tl_checkpoint_1, early_stop, plot_loss_1],
                            verbose=1)

In [ ]:
# Generate predictions
vgg_model.load_weights('tl_model_v1.weights.best.hdf5') # initialize the best trained weights

true_classes = testgen.classes
class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

vgg_preds = vgg_model.predict(testgen)
vgg_pred_classes = np.argmax(vgg_preds, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

vgg_acc = accuracy_score(true_classes, vgg_pred_classes)
print("VGG16 Model Accuracy without Fine-Tuning: {:.2f}%".format(vgg_acc * 100))

In [ ]:
# trying fine tuning
# Reset our image data generators
traingen.reset()
validgen.reset()
testgen.reset()

# Use a smaller learning rate
optim_2 = Adam(lr=0.0001)

# Re-compile the model, this time leaving the last 2 layers unfrozen for Fine-Tuning
vgg_model_ft= create_model(input_shape, n_classes, optim_2, fine_tune=2)

In [ ]:
%%time

plot_loss_2 = PlotLossesCallback()

# Retrain model with fine-tuning
vgg_ft_history = vgg_model_ft.fit(traingen,
                                  batch_size=BATCH_SIZE,
                                  epochs=n_epochs,
                                  validation_data=validgen,
                                  steps_per_epoch=n_steps,
                                  validation_steps=n_val_steps,
                                  callbacks=[tl_checkpoint_1, early_stop, plot_loss_2],
                                  verbose=1)

### 3.2 Save model

In [ ]:
vgg_model.save("/content/gdrive/MyDrive/Image_Classification/model_VGG16.h5")

### 3.3 Test model

In [ ]:
# Generate predictions
vgg_model_ft.load_weights('tl_model_v1.weights.best.hdf5') # initialize the best trained weights

vgg_preds_ft = vgg_model_ft.predict(testgen)
vgg_pred_classes_ft = np.argmax(vgg_preds_ft, axis=1)

In [ ]:
vgg_acc_ft = accuracy_score(true_classes, vgg_pred_classes_ft)
print("VGG16 Model Accuracy with Fine-Tuning: {:.2f}%".format(vgg_acc_ft * 100))
